# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$.
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки.
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вершину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии.

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):

    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обозначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер.

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения узлов дерева (используйте, например, `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признаков и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три разных критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* <underline> purity (остановка, если все объекты в листе относятся к одному классу) </underline>

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
  def __init__(self, min_samples_split=None, max_depth=None, max_leaves_num=None, purity=True):
      # Критерии остановки
      self.min_samples_split = min_samples_split
      self.max_depth = max_depth
      self.max_leaves_num = max_leaves_num
      self.purity = purity

      self.leaves_count = 0
      self.root = None

  def _most_common_label(self, y):
      counter = Counter(y)
      return counter.most_common(1)[0][0]

  def _split(self, X_column, threshold):
      left_idxs = np.argwhere(X_column < threshold).flatten()
      right_idxs = np.argwhere(X_column >= threshold).flatten()

      return left_idxs, right_idxs

  # вычисление информационного прироста
  def _information_gain(self, y, X_column, threshold):
      parent_entropy = entropy(y)

      left_idxs, right_idxs = self._split(X_column, threshold)
      if (len(left_idxs) == 0 or len(right_idxs == 0)):
          return 0;

      n = len(y)
      n_l, n_r = len(left_idxs), len(right_idxs)
      e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
      child_entropy = (n_l / n) * e_l + (n_r/n) * e_r

      return parent_entropy - child_entropy

  def _best_split(self, X, y, feat_idxs):
      best_gain = -1
      split_idx, split_treshold = None, None

      for feat_idx in feat_idxs:
        X_column = X[:, feat_idx]
        thresholds = np.unique(X_column)

        for threshold in thresholds:
            gain = self._information_gain(y, X_column, threshold)
            if (gain > best_gain):
                best_gain = gain
                split_idx = feat_idx
                split_threshold = threshold
      return split_idx, split_treshold

  def _grow_tree(self, X, y, depth=0):
      n_samples, n_features = X.shape
      n_labels = len(np.unique(y))

      if (((depth >= self.max_depth) if (self.max_depth is not None) else False) or
          ((n_labels == 1) if (self.purity == True) else False)or
          ((n_samples < self.min_samples_split) if (self.min_samples_split is not None) else False)):

          leaf_value = self._most_common_label(y)

          return Node(value=leaf_value)

      feat_idxs = np.random.choice(n_features, self.n_features, replace=False)

      best_feature, best_threshold = self._best_split(X, y, feat_idxs)

      left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
      left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
      right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)

      return Node(best_feature, best_threshold, left, right)

  def fit(self, X, y):
      self.n_features = X.shape[1]
      self.root = self._grow_tree(X, y)

  def _traverse_tree(self, x, node):
      if (node.is_leaf_node()):
          return node.value

      if x[node.feature] < node.threshold:
            return self._traverse_tree(x, node.left)
      return self._traverse_tree(x, node.right)

  def predict(self, X):
      return np.array([self._traverse_tree(x, self.root) for x in X])


In [ ]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=None, max_depth=None, max_leaves_num=None, purity=True):
        # Критерии остановки
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.max_leaves_num = max_leaves_num
        self.purity = purity

        self.root = None

    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    # вычисление информационного прироста
    def _information_gain(self, y, X_column, threshold):
        parent_entropy = entropy(y)

        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        left_len, right_len = len(left_idxs), len(right_idxs)
        left_entropy, right_entropy = entropy(y[left_idxs]), entropy(y[right_idxs])
        child_entropy = (left_len / n) * left_entropy + (right_len / n) * right_entropy

        information_gain = parent_entropy - child_entropy
        return information_gain


    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold


    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        if (((depth >= self.max_depth) if (self.max_depth is not None) else False) or
           ((n_labels == 1) if (self.purity == True) else (len(y) >= 1)) or
           ((n_samples < self.min_samples_split) if (self.min_samples_split is not None) else False)):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # разделение на ветви
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(best_feature, best_thresh, left, right)

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.root = self._grow_tree(X, y)

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import sys
sys.setrecursionlimit(100000)

iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)


decisionTree_max_depth = DecisionTree(max_depth=5)
decisionTree_max_depth.fit(X_train, y_train)
predictions_max_depth = decisionTree_max_depth.predict(X_test)
accuracy_max_depth = accuracy_score(y_test, predictions_max_depth)

decisionTree_max_leaves = DecisionTree(max_leaves_num=5)
decisionTree_max_leaves.fit(X_train, y_train)
predictions_max_leaves = decisionTree_max_leaves.predict(X_test)
accuracy_max_leaves = accuracy_score(y_test, predictions_max_leaves)

decisionTree_min_samples_split = DecisionTree(min_samples_split=5)
decisionTree_min_samples_split.fit(X_train, y_train)
predictions_min_samples_split = decisionTree_min_samples_split.predict(X_test)
accuracy_min_samples_split = accuracy_score(y_test, predictions_min_samples_split)

decisionTree_purity = DecisionTree(purity=True)
decisionTree_purity.fit(X_train, y_train)
predictions_purity = decisionTree_purity.predict(X_test)
accuracy_purity = accuracy_score(y_test, predictions_purity)

from tabulate import tabulate
table = [[' ', 'max_depth', 'max_leaves', 'min_samples', 'purity'],
         ['Accuracy', accuracy_max_depth, accuracy_max_leaves, accuracy_min_samples_split, accuracy_purity]]
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

╒══════════╤═════════════╤══════════════╤═══════════════╤══════════╕
│          │   max_depth │   max_leaves │   min_samples │   purity │
╞══════════╪═════════════╪══════════════╪═══════════════╪══════════╡
│ Accuracy │        0.96 │         0.98 │          0.96 │     0.98 │
╘══════════╧═════════════╧══════════════╧═══════════════╧══════════╛


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
import math

class RandomForest:
    def __init__(self, n_trees=10,*, max_tree_depth=None, min_tree_samples_split=2, features_num=None):
        self.n_trees = n_trees
        self.max_tree_depth = max_tree_depth
        self.min_tree_samples_split = min_tree_samples_split
        self.features_num = features_num #k

        self.trees = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        if self.features_num is None:
                        self.features_num = math.ceil(math.sqrt(n_features))

        for i in range(self.n_trees):
            #decisionTree = DecisionTree(min_samples_split=self.min_tree_samples_split, max_depth=self.max_tree_depth)
            decisionTree = DecisionTreeClassifier(criterion='entropy',
                                                  max_depth=self.max_tree_depth,
                                                  min_samples_split=self.min_tree_samples_split,
                                                  max_features=self.features_num)

            idxs = np.random.choice(n_samples, n_samples, replace=True)

            X_train, y_train = X.iloc[idxs], y.iloc[idxs]

            decisionTree.fit(X_train, y_train)

            self.trees.append(decisionTree)

    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        predictions = np.swapaxes(predictions, 0, 1)
        return np.array([self._most_common_label(prediction) for prediction in predictions])

### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес".

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("drive/MyDrive/Colab Notebooks//churn.csv")

Mounted at /content/drive


In [ ]:
print(data)

      RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMemb

In [ ]:
from sklearn.preprocessing import LabelEncoder

data = data.drop(["RowNumber", "Surname", "CustomerId"], axis=1)
data['Gender'] = data['Gender'].replace({'Male': 0, 'Female': 1})

label_encoder = LabelEncoder()
data['Geography'] = label_encoder.fit_transform(data['Geography'])


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop(["Exited"], axis=1, inplace=False),
                                                    data["Exited"],
                                                    train_size=0.7)

In [ ]:
trees_amounts = [5, 10, 15, 20, 25, 30, 35, 40, 45]
min_samples_splits = [2, 3, 4, 5, 6, 7, 8, 9, 10]
max_depths = [5, 10, 15, 20, 25, 30]
features_amounts = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
best_accuracy = -1
best_forest = None
for n_features in features_amounts:
  for trees_num in trees_amounts:
    for min_samples_split in min_samples_splits:
      for max_depth in max_depths:
        forest = RandomForest(n_trees=trees_num, max_tree_depth=max_depth, min_tree_samples_split=min_samples_split, features_num=n_features)
        forest.fit(X_train, y_train)
        prediction = forest.predict(X_test)
        accuracy = accuracy_score(y_test, prediction)
        if accuracy > best_accuracy:
          best_accuracy = accuracy
          best_forest = forest
          best_params = {
              'n_trees' : trees_num,
              'min_tree_samples_split' : min_samples_split,
              'max_depth' : max_depth,
              'features_num' : n_features
          }
print(best_accuracy, best_params)


0.8713333333333333 {'n_trees': 45, 'min_tree_samples_split': 10, 'max_depth': 10, 'features_num': 5}


In [ ]:
testTree = DecisionTree(max_depth = 10, min_samples_split=10) # Эталонное дерево
testTree.fit(X_train, y_train)
# внутренний обход дерева для выяснения важности признаков
root = testTree.root
features_in_order = []
def inner_traverse(tnode):
    if (tnode.feature is None):
      return

    features_in_order.append(tnode.feature)
    inner_traverse(tnode.left)
    inner_traverse(tnode.right)
inner_traverse(root)
print('Топ 3 признака по важности')
for feature in list(dict.fromkeys(features_in_order)):
  print(data.columns.values.tolist()[feature])

Топ 3 признака по важности
Gender
Age
Geography
